In [6]:
import pandas as pd
import json

In [69]:
json_file = open('C:/Users/ramon/OneDrive/INSEAD/auction_proj/full_dataset.json', 'r', encoding='utf-8')
data = json.load(json_file)

In [67]:
if 'box_info' in data[:]:
    print(len(data['box_info'].keys))
else:
    print('cant do that')

cant do that


In [70]:
data

[{'name': ['Aachen, Hans von',
   'Aalto, Alvar',
   'Abbati, Giuseppe',
   'Abdessemed, Adel',
   'Abelenda Zapata, Manuel',
   'Accardi, Carla',
   'Acht, René Charles',
   'Adam, James',
   'Adam, Otto',
   'Adam, Robert',
   'Adams, Charles Partridge',
   'Adams, Clinton',
   'Adams, Mark',
   'Adams, Robert',
   'Adler, Dankmar',
   'Adler, David A.',
   'Adler, Jankel',
   'Adler, Karl-Heinz',
   'Adrian, Marc',
   'Aeppli, Eva',
   'Aertsen, Pieter',
   'Afro',
   'Ahrens, Carl',
   'Aitchison, Craigie',
   'Aitken, Doug',
   'Akers, Benjamin Paul',
   'Albani, Francesco',
   'Albers, Anni',
   'Albers, Josef',
   'Alberti, Leon Battista',
   'Albertinelli, Mariotto',
   'Albrizzi, Enrico',
   'Alechinsky, Pierre',
   'Alexander, John White',
   'Alfaro, Andreu',
   'Alfieri, Attilio',
   'Algardi, Alessandro',
   "Aligny, Théodore Caruelle d'",
   'Alix, Yves',
   'Allar, André Joseph',
   'Allen, Harry Epworth',
   'Allori, Alessandro',
   'Allston, Washington',
   'Alma-Tadem

In [4]:
??pd.read_json

Signature:
pd.read_json(
    path_or_buf=None,
    orient=None,
    typ='frame',
    dtype=None,
    convert_axes=None,
    convert_dates=True,
    keep_default_dates=True,
    numpy=False,
    precise_float=False,
    date_unit=None,
    encoding=None,
    lines=False,
    chunksize=None,
    compression='infer',
)
Source:   
def read_json(
    path_or_buf=None,
    orient=None,
    typ="frame",
    dtype=None,
    convert_axes=None,
    convert_dates=True,
    keep_default_dates=True,
    numpy=False,
    precise_float=False,
    date_unit=None,
    encoding=None,
    lines=False,
    chunksize=None,
    compression="infer",
):
    """
    Convert a JSON string to pandas object.

    Parameters
    ----------
    path_or_buf : a valid JSON str, path object or file-like object
        Any valid string path is acceptable. The string could be a URL. Valid
        URL schemes include http, ftp, s3, and file. For file URLs, a host is
        expected. A local file could be:
        ``file